### Connecting to SQL from Python

In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd

Reading data in Dataframe  
Note : Column names are returned as tuples
https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor

In [2]:
with sqlite3.connect('sqlite_db_pythonsqlite.db') as conn  :
    facilities = conn.execute("SELECT * FROM Facilities")
    print(type(facilities))
    r = facilities.fetchall()
    df_fac = pd.DataFrame(r)
    df_fac.columns = [col[0] for col in facilities.description]

<class 'sqlite3.Cursor'>


In [3]:
df_fac.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [4]:
with sqlite3.connect('sqlite_db_pythonsqlite.db') as conn  :
    q10 = conn.execute("SELECT f.name AS facility, \
        SUM(CASE WHEN b.memid = 0  THEN f.guestcost * b.slots \
        ELSE f.membercost * b.slots END) AS total_revenue \
    FROM Bookings AS b \
    LEFT JOIN Facilities AS f \
    ON f.facid = b.facid \
    GROUP BY f.name \
    HAVING total_revenue < 1000 \
    ORDER BY total_revenue")
    r = q10.fetchall()
    df_fac = pd.DataFrame(r)
    df_fac.columns = [col[0] for col in q10.description]
    print(df_fac)


        facility  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order


In [5]:
with sqlite3.connect('sqlite_db_pythonsqlite.db') as conn  :
    q11 = conn.execute("SELECT m.surname,m.firstname , ( r.surname || ' ' || r.firstname ) AS referred_by \
    FROM Members AS m \
    LEFT JOIN Members AS r \
    ON m.recommendedby = r.memid \
    WHERE m.recommendedby > 0 \
    ORDER BY m.surname, m.firstname")
    r = q11.fetchall()
    df_fac = pd.DataFrame(r)
    df_fac.columns = [col[0] for col in q11.description]
    print(df_fac.head(15))


      surname firstname        referred_by
0       Bader  Florence    Stibbons Ponder
1       Baker      Anne    Stibbons Ponder
2       Baker   Timothy     Farrell Jemima
3      Boothe       Tim         Rownam Tim
4     Butters    Gerald       Smith Darren
5      Coplin      Joan      Baker Timothy
6     Crumpet     Erica        Smith Tracy
7        Dare     Nancy    Joplette Janice
8     Genting   Matthew     Butters Gerald
9        Hunt      John  Purview Millicent
10      Jones     David    Joplette Janice
11      Jones   Douglas        Jones David
12   Joplette    Janice       Smith Darren
13  Mackenzie      Anna       Smith Darren
14       Owen   Charles       Smith Darren


Q12: Find the facilities with their usage by member, but not guests

In [6]:
with sqlite3.connect('sqlite_db_pythonsqlite.db') as conn  :
    q12 = conn.execute("SELECT ( m.surname || ' ' || m.firstname ) AS member_name, \
    f.name AS facility, SUM( b.slots ) AS total_slots \
    FROM Bookings AS b \
    LEFT JOIN Facilities AS f ON b.facid = f.facid \
    LEFT JOIN Members AS m ON b.memid = m.memid \
    WHERE b.memid !=0 \
    GROUP BY b.memid, f.name \
    ORDER BY member_name, total_slots DESC")
    r = q12.fetchall()
    df_fac = pd.DataFrame(r)
    df_fac.columns = [col[0] for col in q12.description]
    print(df_fac.head(15))
    

       member_name         facility  total_slots
0   Bader Florence     Table Tennis           86
1   Bader Florence    Snooker Table           66
2   Bader Florence  Badminton Court           27
3   Bader Florence   Tennis Court 2           24
4   Bader Florence       Pool Table           23
5   Bader Florence     Squash Court            4
6   Bader Florence   Massage Room 2            4
7   Bader Florence   Tennis Court 1            3
8       Baker Anne   Tennis Court 2          114
9       Baker Anne     Squash Court          110
10      Baker Anne  Badminton Court           30
11      Baker Anne   Tennis Court 1           18
12      Baker Anne       Pool Table           12
13      Baker Anne   Massage Room 1            6
14      Baker Anne   Massage Room 2            4


Q13: Find the facilities usage by month, but not guests 

In [7]:
with sqlite3.connect('sqlite_db_pythonsqlite.db') as conn  :
    q13 = conn.execute("SELECT f.name AS facility, strftime('%m', starttime) AS month,SUM( b.slots ) AS total_slots\
    FROM Bookings AS b \
    LEFT JOIN Facilities AS f ON b.facid = f.facid \
    WHERE b.memid !=0 \
    GROUP BY f.name, strftime('%m', starttime) \
    ORDER BY f.name")
    r = q13.fetchall()
    df_fac = pd.DataFrame(r)
    df_fac.columns = [col[0] for col in q13.description]
    print(df_fac.head(15))



           facility month  total_slots
0   Badminton Court    07          165
1   Badminton Court    08          414
2   Badminton Court    09          507
3    Massage Room 1    07          166
4    Massage Room 1    08          316
5    Massage Room 1    09          402
6    Massage Room 2    07            8
7    Massage Room 2    08           18
8    Massage Room 2    09           28
9        Pool Table    07          110
10       Pool Table    08          303
11       Pool Table    09          443
12    Snooker Table    07          140
13    Snooker Table    08          316
14    Snooker Table    09          404
